# Gradio with OpenAI API (via OpenRouter)

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/02/notebooks/gradio.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/02/notebooks/gradio.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Install required packages

In [ ]:
!uv add gradio==5.49.1 openai

## Set the OpenRouter API Key from Colab Secrets

In [ ]:
from google.colab import userdata
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

## (Or grab the OpenRouter API key if running locally)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")

## Initialize the OpenAI client

In [ ]:
import openai

# Initialize OpenAI client
client = openai.OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY,
)

## Example 1: Basic Gradio interface

In [7]:
import gradio as gr

def image_classifier(inp):
    return {'cat': 0.3, 'dog': 0.7}

demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label")
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## Example 2: Basic chat interface with conversation history

In [8]:
import gradio as gr

def chat_with_history(message, history):
    # Add current message
    messages = history + [{"role": "user", "content": message}]
    
    # Get response from API
    response = client.chat.completions.create(
        model='openai/gpt-5.2-chat',
        messages=messages,
    )
    
    return response.choices[0].message.content

# Create a chat interface
demo = gr.ChatInterface(
    fn=chat_with_history,
    title="Basic Chat with Conversation History",
    type="messages"
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


## Example 3: Streaming chat interface

In [9]:
import gradio as gr

def chat_with_streaming(message, history):
    messages = history + [{"role": "user", "content": message}]
    
    # Stream the response
    stream = client.chat.completions.create(
        model='openai/gpt-5.2-chat',
        messages=messages,
        stream=True,
    )
    
    response_text = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            token = chunk.choices[0].delta.content
            response_text += token
            yield response_text

# Create streaming chat interface
demo = gr.ChatInterface(
    fn=chat_with_streaming,
    title="AI Chat with Streaming",
    type="messages"
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
